# ASI Hunting - SecurityEvent Template

We would like to guide you though your threat hunting efforts:
    1. Prepare the shell;
    2. Enter values;
    3. Execute queries;
    4. Analyze data.


## 0. Retrieve Log Analytics Information

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from IPython.display import HTML

from utils import config_reader as mod
workspaceId = mod.config_reader.read_config_values("../config.json")[3]

######################################################################
your_workspace_Name = mod.config_reader.read_config_values("../config.json")[4]
print("Your Log Analytic Workspace: " + your_workspace_Name)
######################################################################

## 1. Install KQL Magic for Log Analytics

In [ ]:
# run once unless upgrade
!pip install Kqlmagic --no-cache-dir --upgrade

In [ ]:
%reload_ext Kqlmagic

If you need any help for kql magic     --> %kql --help "help" -popup_window
If you need any help for Log Analytics --> %kql --help "LogAnalytics"

## 2. Log into Log Analytics

In [ ]:
%kql loganalytics://code;workspace=workspaceId;alias="asidb"

## 3. Filter Data

1. Enter How many days looking back

In [ ]:
enteredDaysBack = input()

2. Select one data source that you want to work on

In [ ]:
dbSchema = %kql --schema "asidb@loganalytics"
tables = list(dbSchema.keys())
selectedTable = widgets.Dropdown(options=tables, value=tables[24],description='Data Table:')
display(selectedTable)

3. Select features

In [ ]:
columns = list(dbSchema[selectedTable.value])
columns.sort()
selectedFeatures = widgets.SelectMultiple(options=columns, value=[],description='Data Columns:')
display(selectedFeatures)

In [ ]:
print(selectedFeatures.value)

4. Pick values for selected features

In [ ]:
%kql {selectedTable.value} | where TimeGenerated >= ago({enteredDaysBack}d) | take 100
if len(_kql_raw_result_) > 0:
    df = _kql_raw_result_.to_dataframe()
else:
    df = None
    print('no data')

In [ ]:
def select_filter_value(df, column):
    df.columns = df.columns.str.strip()
    columnData = df[column].drop_duplicates().sort_values()
    selectedType = widgets.Dropdown(options=columnData, description=column)
    display(selectedType)
    return selectedType.value

In [ ]:
selectedPairs = {}
for index, column in enumerate(selectedFeatures.value):
    #print(df[column].drop_duplicates().sort_values())
    print(selectedFeatures.value[index] + ": ")
    selectedPairs[selectedFeatures.value[index]] = input()
    
whereClause = ""
for index, column in enumerate(selectedFeatures.value):
    whereClause += " | where " + selectedFeatures.value[index] + " contains '" + selectedPairs[selectedFeatures.value[index]].strip() + "'"

print(whereClause)

## 4. KQL operations, results are returned in Pandas DataFrame

In [ ]:
%kql {selectedTable.value} | where TimeGenerated >= ago({enteredDaysBack}d) {whereClause} | take 100
if len(_kql_raw_result_) > 0:
    df = _kql_raw_result_.to_dataframe()
else:
    df = None
    print('no data')

## 5. Data Analysis

In [ ]:
if df is not None:
    dfMachine = df.groupby(['AccountType', 'Computer'])['AccountType'].count()
    dfMachine.plot(kind='bar',x='AccountType')

In [ ]:
df

In [ ]:
# Load queries from GitHub
import requests
import json
#url = "https://raw.githubusercontent.com/ASIJupyter/ASINotebooks/master/queries/Anomalous%20Azure%20AD%20apps%20based%20on%20authentication%20location.json"
url = "https://raw.githubusercontent.com/ASIJupyter/ASINotebooks/master/queries/Uncommon%20processes.json"
#url = "https://raw.githubusercontent.com/ASIJupyter/ASINotebooks/master/queries/Script%20usage%20summary.json";
response = requests.get(url)
resJson = json.loads(response.text)
if response != None:
    try:
        print(resJson["query"])
    except:
        print(type("type: " + response.text))

In [ ]:
#%kql {resJson["query"]}

%kql -query=resJson["query"]

In [ ]:
_

In [ ]:
print(_kql_raw_result_.parametrized_query)

In [ ]:
def constructLAUrl():
    urlTemplate = "https://ms.portal.azure.com/?feature.showassettypes=Microsoft_Azure_Security_Insights_SecurityInsightsDashboard#blade/Microsoft_Azure_Security_Insights/MainMenuBlade/8/subscriptionId/{0}/resourceGroup/{1}/workspaceName/{2}"
    url = urlTemplate.format(mod.config_reader.read_config_values("../config.json")[1], mod.config_reader.read_config_values("../config.json")[2], mod.config_reader.read_config_values("../config.json")[4])

    s  = '<html><body> '
    s += '<a href="' + url + '" target="_blank">Back to LA</a>'
    s += '</body></html>'
    return s

In [ ]:
# Open  browser to LA
HTML(constructLAUrl())

In [ ]:
# output data to csv
df.to_csv('mydata.csv')

In [ ]:
# call bookmark API to put
bookmark_id = "E19A22A1-78FF-473C-89B1-6C2DA3235DA3"
import requests
import json
url = "https://resourceprovider.westus2.cloudapp.azure.com/subscriptions/9428a5ff-0f33-4c1d-b811-ed0e96c14d4a/resourceGroups/ian/providers/Microsoft.OperationalInsights/workspaces/test/providers/Microsoft.SecurityInsights/HuntingBookmarks/" + bookmark_id + "?api-version=1.0"

bm = '{"BookmarkId": "' + bookmark_id + '","BookmarkName": "bookmark1","Notes": "somethig here","Query": "SecurityEvent ","QueryResult": "a record"}'
bmJson = json.loads(bm)

print(bm)
print(url)
#print(type(bmJson))
response = requests.put(url, bmJson)
print(response.text)